## Setting the environment


In [1]:
!pip install myosuite==2.8.3 --quiet
!pip install "stable-baselines3[extra]" --quiet
!pip install tqdm --quiet
!pip install mujoco==3.1.2 --quiet
!pip install sk-video --quiet
!pip install wget --quiet
import os
if 'google.colab' in str(get_ipython()):
    current_dir = os.getcwd()
    print(current_dir)
    if current_dir == '/content':
        !git clone https://github.com/ttktjmt/myochallenge-neuroflex.git
        os.chdir('myochallenge-neuroflex')

os.environ['MUJOCO_GL'] = 'egl'
import mujoco
import wget
from IPython.display import HTML
from base64 import b64encode
import myosuite
from myosuite.utils import gym
import skvideo.io
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
from tqdm.notebook import tqdm
import importlib
import envs
import warnings
from datetime import datetime
from IPython.display import clear_output, Image, display
import PIL.Image
import numpy as np
# the descrepancy of numpy version between the environment where the policy is trained and loaded can cause irrelevant issues
print(f"numpy version: {np.__version__}")
import os
import skvideo.io

libEGL warning: failed to open /dev/dri/renderD128: Permission denied

libEGL warning: NEEDS EXTENSION: falling back to kms_swrast


pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
MyoSuite:> Registering Myo Envs
numpy version: 2.1.3


## Define Functions

In [2]:
def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")

## Download baseline from Google Drive

In [3]:
file_id = '168S3oTBmXuf23zDDGMM3tCJ3HBnjYhsM'
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'
target_file = 'baseline.zip'  # Change the file name and extension as needed

os.system(f'wget --no-check-certificate "{download_url}" -O {target_file}')

if os.path.exists(target_file):
    print(f'Successfully downloaded {target_file}')
else:
    print('Failed to download the file')

--2024-11-22 13:15:19--  https://drive.google.com/uc?export=download&id=168S3oTBmXuf23zDDGMM3tCJ3HBnjYhsM
Resolving drive.google.com (drive.google.com)... 172.217.31.142, 2404:6800:4004:825::200e
Connecting to drive.google.com (drive.google.com)|172.217.31.142|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=168S3oTBmXuf23zDDGMM3tCJ3HBnjYhsM&export=download [following]
--2024-11-22 13:15:20--  https://drive.usercontent.google.com/download?id=168S3oTBmXuf23zDDGMM3tCJ3HBnjYhsM&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.196.129, 2404:6800:4004:80a::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.196.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3167662 (3.0M) [application/octet-stream]
Saving to: ‘baseline.zip’

     0K .......... .......... .......... .......... ..........  1%  626K 

Successfully downloaded baseline.zip


. .......... .......... .......... .......... 59% 4.57M 0s
  1850K .......... .......... .......... .......... .......... 61% 17.2M 0s
  1900K .......... .......... .......... .......... .......... 63% 27.1M 0s
  1950K .......... .......... .......... .......... .......... 64% 13.5M 0s
  2000K .......... .......... .......... .......... .......... 66% 9.14M 0s
  2050K .......... .......... .......... .......... .......... 67% 6.96M 0s
  2100K .......... .......... .......... .......... .......... 69% 10.5M 0s
  2150K .......... .......... .......... .......... .......... 71% 11.1M 0s
  2200K .......... .......... .......... .......... .......... 72% 9.24M 0s
  2250K .......... .......... .......... .......... .......... 74% 9.21M 0s
  2300K .......... .......... .......... .......... .......... 75% 7.06M 0s
  2350K .......... .......... .......... .......... .......... 77% 9.61M 0s
  2400K .......... .......... .......... .......... .......... 79% 8.33M 0s
  2450K .......... ..........

## Environment

In [3]:
importlib.reload(envs)
env_name = 'myoChallengeBimanual-v0'
env = gym.make(env_name)

    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment CustomBimanualEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/myosuite/envs/myo/myochallenge/bimanual_v0.py:432: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.manip_joint_range = np.arange(model.joint("manip_object/freejoint").qposadr,
/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/myosuite/envs/myo/myochallenge/bimanual_v0.py:432: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before perfor

In [4]:
checkpoint_callback = CheckpointCallback(save_freq=50000, save_path='./logs/', name_prefix='ppo_model')
# model_file_path = "baseline.zip"
# model_file_path = "logs/ppo_model_400000_steps.zip"
model_file_path = "agent/model.zip"

if os.path.exists(model_file_path):
    try:
        model = PPO.load(model_file_path, env, device='cpu')
        print("Model loaded successfully!")
    except Exception as e:
        print(f"Error loading model: {e}")
else:
    print("Model file does not exist. Please train the model first.")

env.reset()
env.seed(27) #setting a seed for the env
tb_log_name = "ppo_training"

/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:165: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  deserialized_object = cloudpickle.loads(base64_object)
/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:165: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discourag

Model loaded successfully!


/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.seed to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.seed` for environment variables or `env.get_wrapper_attr('seed')` that will search the reminding wrappers.
  logger.warn(


## Train

In [7]:
model.learn(total_timesteps=500000, callback=checkpoint_callback, tb_log_name=tb_log_name)

/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


KeyboardInterrupt: 

## Visualize

In [6]:
warnings.filterwarnings("ignore", message=".*tostring.*is deprecated.*")

movie = True  # we want to save a video
frames = []
# view = 'front'
# obj_qpos_list = []
release_threshold = 0.08
released_step = -1
waiting_step1 = 100
waiting_step2 = 150
waiting_step3 = 200

for _ in range(10):
    ep_rewards = 0
    done = False
    obs = env.reset()
    _frames = []
    for step in range(300):
        obs_dict = env.obs_dict
        obs = env.obsdict2obsvec(obs_dict, env.obs_keys)[1]
        action, _ = model.predict(obs, deterministic=True)
        # myoHand control
        action[30] = 1
        obj_xpos = obs_dict['object_qpos'][0]
        if obj_xpos > release_threshold and released_step == -1:
            released_step = step

        if step - released_step > waiting_step1:
            action[32:40] = 0
            action[40:49] = 1

        # MPL control
        action[-17:] = np.array([-0.65001469, 1., -0.23187843, 0.59583695, 0.92356688, -0.16,
                                -0.28, -0.88, 0.25, -0.846, -0.24981132, -0.91823529,
                                -0.945, -0.925, -0.929, -0.49, -0.18])
        if step - released_step > waiting_step2:
            action[-17:] = np.array([-0.4199236, 1., -0.9840558, 0.35299219, 0.92356688, 0.02095238,
                                    -0.28, -0.88, 0.25, -0.846, -0.24981132, -0.91823529,
                                    -0.945, -0.925, -0.929, -0.49, -0.918])
        if step - released_step > waiting_step3:
            action[-17:] = np.array([-0.4199236, 1., -0.9840558, 0.35299219, 0.3910828, 0.02095238,
                                    -0.28, -0.88, 0.25, -0.846, -0.24981132, -0.91823529,
                                    -0.945, -0.925, -0.929, -0.49, -0.918])

        obs, reward, done, info, info_2 = env.step(action)
        ep_rewards += reward  # Accumulate rewards for the episode
        if movie:
            geom_1_indices = np.where(env.sim.model.geom_group == 1)
            env.sim.model.geom_rgba[geom_1_indices, 3] = 0
            frame = env.sim.renderer.render_offscreen(width=400, height=400, camera_id=1)

            # Clear previous output and display new frame
            # obj_qpos_list.append(obs_dict['object_qpos'])

            # Convert numpy array to PIL Image
            # clear_output(wait=True)
            # pil_image = PIL.Image.fromarray(frame)
            # display(pil_image)

            _frames.append(frame)
    
    print(f"{ep_rewards = }")
    if ep_rewards > 0:
        frames.extend(_frames)

env.close()

# Save video
os.makedirs('videos', exist_ok=True)
current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
skvideo.io.vwrite(f'videos/test_policy_{current_date}.mp4',
                  np.asarray(frames),
                  inputdict={'-r': '100'},
                  outputdict={"-pix_fmt": "yuv420p"})
show_video(f'videos/test_policy_{current_date}.mp4')

/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/myosuite/envs/myo/myochallenge/bimanual_v0.py:368: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.sim.model.geom(self.obj_gid).size = self.obj_size0 * obj_scales
/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obs_dict to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.obs_dict` for environment variables or `env.get_wrapper_attr('obs_dict')` that will search the reminding wrappers.
  logger.warn(
/home/ta747375ki/myochallenge-neuroflex/venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obsdict2obsvec to get variables from other wrappers is deprecated and will be

KeyboardInterrupt: 